<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifiers/TAPP/CPU_TAPP_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
# ! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [2]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 09:20:27] [setup] RAM Tracking...
[codecarbon INFO @ 09:20:27] [setup] GPU Tracking...
[codecarbon INFO @ 09:20:27] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:20:27] [setup] CPU Tracking...
[codecarbon WARNING @ 09:20:27] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 09:20:28] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 09:20:28] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 09:20:28] >>> Tracker's metadata:
[codecarbon INFO @ 09:20:28]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 09:20:28]   Python version: 3.10.12
[codecarbon INFO @ 09:20:28]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 09:20:28]   Available RAM : 12.675 GB
[codecarbon INFO @ 09:20:28]   CPU count: 2
[codecarbon INFO @ 09:20:28]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 09:20:28]   GPU c

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Manual Inputs


In [6]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
train_file="policy_classification_train.json"
test_file="policy_classification_test.json"


### Define labels ##################
label_names= ['TargetLabel','ActionLabel','PlansLabel','PolicyLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "BAAI/bge-base-en-v1.5"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "TAPP-multilabel-bge"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=7.40e-5
batch_size = 16
num_train_epochs= 7
weight_decay=0.05
warmup_steps = 200
gradient_accumulation_steps = 1

# Automated

## Dataset

In [7]:
# Load Dataset from Hugging Face
data_files = {"train": train_file, "test": test_file}
dataset = load_dataset(repo_id, data_files=data_files)

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = dataset['train'].to_pandas()
test_df =dataset['test'].to_pandas()

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{0: 'ActionLabel', 1: 'PlansLabel', 2: 'PolicyLabel', 3: 'TargetLabel'}


In [8]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 10031
0 . ActionLabel : 5416
1 . PlansLabel : 2140
2 . PolicyLabel : 1396
3 . TargetLabel : 2911

 Classes Representation in Test Dataset: 932
0 . ActionLabel : 513
1 . PlansLabel : 198
2 . PolicyLabel : 122
3 . TargetLabel : 256


In [9]:
# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}

for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))
    pos = sum(train_df[label])
    neg = len(train_df) - sum(train_df[label])
    positive_weights[label] = max(neg/pos, 1.1)
    negative_weights[label] = min(pos/neg,1.0)
print(positive_weights)
print(negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . ActionLabel : 5416
1 . PlansLabel : 2140
2 . PolicyLabel : 1396
3 . TargetLabel : 2911
{'ActionLabel': 1.1, 'PlansLabel': 3.6873831775700934, 'PolicyLabel': 6.185530085959885, 'TargetLabel': 2.4458948814840262}
{'ActionLabel': 1.0, 'PlansLabel': 0.2711950323152959, 'PolicyLabel': 0.161667631731326, 'TargetLabel': 0.4088483146067416}


`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [10]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at BAAI/bge-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Training data: 10031
Validation data: 932


Map:   0%|          | 0/10031 [00:00<?, ? examples/s]

[codecarbon INFO @ 09:21:43] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 09:21:43] Energy consumed for all GPUs : 0.000234 kWh. Total GPU Power : 17.164338172226877 W
[codecarbon INFO @ 09:21:43] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:21:43] 0.001218 kWh of electricity used since the beginning.


Map:   0%|          | 0/932 [00:00<?, ? examples/s]

Map:   0%|          | 0/10031 [00:00<?, ? examples/s]

Map:   0%|          | 0/932 [00:00<?, ? examples/s]

## Metrics

In [12]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [13]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "cosine",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

## Custom Trainer

In [14]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights, reduction='mean')
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Training

In [15]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

[codecarbon INFO @ 09:21:54] [setup] RAM Tracking...
[codecarbon INFO @ 09:21:54] [setup] GPU Tracking...
[codecarbon INFO @ 09:21:54] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:21:54] [setup] CPU Tracking...
[codecarbon WARNING @ 09:21:54] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 09:21:56] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 09:21:56] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 09:21:56] >>> Tracker's metadata:
[codecarbon INFO @ 09:21:56]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 09:21:56]   Python version: 3.10.12
[codecarbon INFO @ 09:21:56]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 09:21:56]   Available RAM : 12.675 GB
[codecarbon INFO @ 09:21:56]   CPU count: 2
[codecarbon INFO @ 09:21:56]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 09:21:56]   GPU c

In [ ]:
multi_trainer.train()

In [17]:
tracker.stop()

[codecarbon INFO @ 10:42:33] Energy consumed for RAM : 0.006493 kWh. RAM Power : 4.753046035766602 W
[codecarbon INFO @ 10:42:33] Energy consumed for all GPUs : 0.093261 kWh. Total GPU Power : 33.58471480397675 W
[codecarbon INFO @ 10:42:33] Energy consumed for all CPUs : 0.058116 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:42:33] 0.157870 kWh of electricity used since the beginning.


0.07145529177313927

In [18]:
multi_trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1709630528.f0a18e2da3f8.1538.0:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/TAPP-multilabel-bge/commit/5765da489a7129e0bc61a5ce685608b52a50eaef', commit_message='End of training', commit_description='', oid='5765da489a7129e0bc61a5ce685608b52a50eaef', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
log_results = multi_trainer.state.log_history.copy()
logs = []
while len(log_results)>1:

    a = log_results[0]
    b = log_results[1]
    logs.append({**a,**b})
    log_results.pop(0)
    log_results.pop(0)

results_df = pd.DataFrame(logs)
results_df

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_precision-micro,eval_precision-samples,eval_precision-weighted,eval_recall-micro,eval_recall-samples,eval_recall-weighted,eval_f1-micro,eval_f1-samples,eval_f1-weighted,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.7161,4.688006,0.000072,1.0,627,0.632245,0.593108,0.637339,0.627373,0.821855,0.783262,0.821855,0.688992,0.672823,0.700018,22.7645,40.941,2.592
1,0.4549,2.674692,0.000063,2.0,1254,0.541975,0.663904,0.689109,0.704947,0.808999,0.768419,0.808999,0.729305,0.704787,0.740856,22.7454,40.975,2.594
2,0.2599,6.364895,0.000048,3.0,1881,0.696589,0.735415,0.739628,0.734604,0.821855,0.784514,0.821855,0.776236,0.742469,0.771314,22.8198,40.842,2.585
3,0.1405,25.174522,0.000031,4.0,2508,0.753023,0.756915,0.749374,0.756930,0.829201,0.789878,0.829201,0.791411,0.750506,0.790463,22.7371,40.990,2.595
4,0.0681,0.426939,0.000015,5.0,3135,0.823394,0.759599,0.753487,0.759906,0.835629,0.794528,0.835629,0.795802,0.754593,0.795318,22.7678,40.935,2.591
5,0.0291,1.519928,0.000004,6.0,3762,0.884944,0.777300,0.764038,0.777617,0.830119,0.788984,0.830119,0.802842,0.759672,0.802743,22.7455,40.975,2.594
6,0.0147,2.823064,0.000000,7.0,4389,0.921702,0.777207,0.764396,0.775577,0.832874,0.792024,0.832874,0.804078,0.760924,0.802881,22.7334,40.997,2.595


# Predict and Evaluate

## Trainer

In [21]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>thresh).bool()
y_true = y_true.bool()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [22]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

ActionLabel
[[324  95]
 [ 60 453]] 

PlansLabel
[[666  68]
 [ 70 128]] 

PolicyLabel
[[773  37]
 [ 24  98]] 

TargetLabel
[[616  60]
 [ 28 228]] 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ActionLabel,0.826642,0.883041,0.853911,513.0
1,PlansLabel,0.653061,0.646465,0.649746,198.0
2,PolicyLabel,0.725926,0.803279,0.762646,122.0
3,TargetLabel,0.791667,0.890625,0.838235,256.0
4,micro avg,0.777207,0.832874,0.804078,1089.0
5,macro avg,0.749324,0.805852,0.776135,1089.0
6,weighted avg,0.775577,0.832874,0.802881,1089.0
7,samples avg,0.764396,0.792024,0.760924,1089.0
